# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI can generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses reduce content creation costs and increase efficiency.
2. **Image and Video Generation**: Generative AI can create realistic images and videos for marketing, advertising, and entertainment purposes. For example, it can be used to generate product images, animations, or even entire movies.
3. **Chatbots and Virtual Assistants**: AI-powered chatbots can provide 24/7 customer support, answer frequently asked questions, and help with simple inquiries. They can also be used in virtual assistants for tasks such as scheduling appointments and sending reminders.
4. **Predictive Analytics and Forecasting**: Generative AI can analyze large datasets to make predictions about future trends and patterns. This can help businesses make informed decisions abo

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [8]:
!pip install ollama


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI (Artificial Intelligence) has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can generate high-quality content such as articles, blog posts, social media posts, and even entire books. This technology is particularly useful for businesses that need to produce large amounts of content quickly and efficiently.
2. **Image and Video Generation**: Generative AI can create realistic images and videos that are indistinguishable from those created by humans. This technology has applications in fields such as advertising, product design, and film production.
3. **Chatbots and Virtual Assistants**: Generative AI-powered chatbots can generate human-like responses to customer inquiries, providing a more personalized experience for customers.
4. **Predictive Maintenance**: Generative AI can analyze sensor data from machines and predict when maintenance is required, reducing downtime and increasing efficiency.


## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [14]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [20]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [21]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [24]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [25]:
summarize("https://edwarddonner.com")

"# Website Summary\nThe website appears to be a personal blog maintained by Edward Donner, a co-founder and CTO of Nebula.io. The site's main content is about artificial intelligence (AI) and large language models (LLMs).\n\n## News and Announcements\n### Latest Posts:\n* **Mastering AI and LLM Engineering – Resources** (published November 13, 2024): A resource post sharing materials for learning AI and LLM engineering.\n* **From Software Engineer to AI Data Scientist – resources** (published October 16, 2024): A resource post sharing materials for transitioning from a software engineer to an AI data scientist.\n* **Outsmart LLM Arena – a battle of diplomacy and deviousness** (published June 26, 2024): An article introducing the Outsmart LLM Arena, where LLMs compete in a battle of diplomacy and deviousness.\n* **Choosing the Right LLM: Toolkit and Resources** (published August 6, 2024): A resource post sharing tools and resources for choosing the right LLM.\n\n### No recent news annou

In [26]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [27]:
display_summary("https://www.nisantasi.edu.tr/")

İstanbul Nişantaşı Üniversitesi, bir Türkiye'deki prestijli ve tanınmış araştırma ve eğitim kuruluştur. Üniversite, öğrencilerinin çeşitli alanlarda akademik ve kültürel faaliyetlere katılımını desteklemek için 다양한 programlara sahiptir.

Üniversitenin ana kampüsü Kağıthane'de yer almaktadır ve NeoTech Campus adıyla bilinen modern bir kampüs yapısı bulunmaktadır.

İstediğiniz información için lütfen aşağıdaki linkleri kullanabilirsiniz:

* https://www.niu.edu.tr/
* https://www.niu.edu.tr/en
* https://niu.mynet.com.tr/

Bu web siteleri, üniversitenin resmi internet siteleri olup, öğrenciler ve interested individuals için birçok bilgi ve informatie sunmaktadır.

Üniversite hakkında daha fazla bilgi almak için lütfen aşağıdaki linkleri kullanabilirsiniz:

* https://www.niu.edu.tr/en/About-NIU
* https://www.niu.edu.tr/tr/Hakikat-Hakkımsızlık-ve-Özgürlük-İçin
* https://www.niu.edu.tr/tr/Universitemiz-Sözlük

Bu web siteleri, üniversite hakkında daha fazla bilgi sunmaktadır ve öğrenciler ve interested individuals için birçok bilgi ve informatie sağlar.